# 🗺️ Planners (with FunctionCallingStepwise planner)

### Initialize the kernel

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.15.0"
#r "nuget: Microsoft.SemanticKernel.Abstractions, 1.15.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.15.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.15.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.15.0-preview"
#r "nuget: Microsoft.SemanticKernel.Planners.OpenAI, 1.15.0-preview"

#!import Secrets.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var kernel = Kernel.CreateBuilder()
    // .AddAzureOpenAIChatCompletion(
    //     deploymentName: Secrets.DeploymentName,
    //     endpoint: Secrets.Endpoint,
    //     apiKey: Secrets.ApyKey)
    .AddOpenAIChatCompletion(
        modelId: Secrets.OpenAIModel,
        apiKey: Secrets.OpenAIApiKey)    
    .Build();

Installed Packages Microsoft.SemanticKernel, 1.15.0 Microsoft.SemanticKernel.Abstractions, 1.15.0 Microsoft.SemanticKernel.Planners.Handlebars, 1.15.0-preview Microsoft.SemanticKernel.Planners.OpenAI, 1.15.0-preview Microsoft.SemanticKernel.Plugins.Core, 1.15.0-alpha Microsoft.SemanticKernel.Plugins.Web, 1.15.0-alpha

### 🔌 Let's add a native plugin for use by the planner

In [2]:
#!import Helpers/PrintHelper.cs
#!import Plugins/MotorPlugin.cs

kernel.Plugins.AddFromType<MotorPlugin>();

PrintHelper.PrintAllPluginFunctions(kernel);

****** Registered 🔌 Plugins and 📦 Functions ******
🔌 MotorPlugin
   📦 /Backward: Moves the car backward.
🔌 MotorPlugin
   📦 /Forward: Moves the car forward.
🔌 MotorPlugin
   📦 /Stop: Stops the car.
🔌 MotorPlugin
   📦 /TurnLeft: Turns the car anticlockwise.
🔌 MotorPlugin
   📦 /TurnRight: Turns the car clockwise.


### 📋 Prepare kernel arguments

In [3]:
var goal = "You have a tree in front of the car. Avoid it.";
var kernelArguments = new KernelArguments()
{
    ["input"] = goal,
    ["commands"] = "go forward, go backward, turn left, turn right, and stop"
};

### Refine the goal

In [4]:
var commandsPlugin = kernel.CreatePluginFromPromptDirectory(System.IO.Path.Combine(System.IO.Directory.GetCurrentDirectory(), "Plugins", "CommandsPlugin"), "CommandsPlugin");
kernel.Plugins.Add(commandsPlugin);
var semanticFunction = commandsPlugin["ExtractBasicCommands"];
var response = await kernel.InvokeAsync(semanticFunction, kernelArguments);
var refinedGoal = response.GetValue<string>();

kernelArguments["input"] = refinedGoal;

display(refinedGoal)

- Go forward (until the car is near the tree)
- Stop
- Turn right
- Go forward (until the car has bypassed the tree)
- Stop
- Turn left
- Go forward (until the car is back on its original path)
- Stop

### 🗺️🧠 Generate FunctionCallingStepwise Plan

This planner is different from the handlebarsplanner in that it doesn't generate a plan ahead of time, and simply progresses towards its goal.

### 🏃 Let's see it run

In [5]:
using Microsoft.SemanticKernel.Planning;

#pragma warning disable SKEXP0060

var functionCallingStepwisePlannerOptions = new FunctionCallingStepwisePlannerOptions
{
    MaxIterations = 30
};

functionCallingStepwisePlannerOptions.ExcludedPlugins.Add("CommandsPlugin");

PrintHelper.PrintAllPluginFunctions(kernel);

****** Registered 🔌 Plugins and 📦 Functions ******
🔌 MotorPlugin
   📦 /Backward: Moves the car backward.
🔌 MotorPlugin
   📦 /Forward: Moves the car forward.
🔌 MotorPlugin
   📦 /Stop: Stops the car.
🔌 MotorPlugin
   📦 /TurnLeft: Turns the car anticlockwise.
🔌 MotorPlugin
   📦 /TurnRight: Turns the car clockwise.
🔌 CommandsPlugin
   📦 /ExtractBasicCommands: Break down complex actions into basic motor commands.
      📥 Params:
       • input: Complex action to be break down into basic motor commands. (default: '')
       • commands: The basic motor commands to choose from. (default: '')


In [6]:
#pragma warning disable SKEXP0060

var planner = new FunctionCallingStepwisePlanner(functionCallingStepwisePlannerOptions);
var plannerResult = await planner.ExecuteAsync(kernel, refinedGoal);

display(plannerResult.FinalAnswer)

🡲
·
🡶
🡲
·
🡵
🡲
·


1. Move the car forward.
2. Stop the car.
3. Turn the car right.
4. Move the car forward.
5. Stop the car.
6. Turn the car left.
7. Move the car forward.
8. Stop the car.

In [ ]:
#pragma warning disable SKEXP0060

functionCallingStepwisePlannerOptions.ExcludedPlugins.Add("CommandsPlugin");
var planner = new FunctionCallingStepwisePlanner(functionCallingStepwisePlannerOptions);
var plannerResult = await planner.ExecuteAsync(kernel, refinedGoal);

display(plannerResult.FinalAnswer)